<a href="https://colab.research.google.com/github/pj0620/google-colab-notebooks/blob/main/acca_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Common

In [16]:
!pip install boto3 awscli
!pip install pandas

import os
import boto3
import pandas as pd

In [13]:
from google.colab import userdata

os.environ['AWS_ACCESS_KEY_ID'] = userdata.get('AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get('AWS_SECRET_ACCESS_KEY')
os.environ['AWS_DEFAULT_REGION'] = userdata.get('AWS_DEFAULT_REGION')

In [20]:
# Initialize a session using Amazon DynamoDB
dynamodb = boto3.resource('dynamodb')

# Select your DynamoDB table
table = dynamodb.Table('events-table-prod')

# Perform a scan operation with a limit of 10 items
response = table.scan(Limit=10)
items = response['Items']
df = pd.DataFrame(items)
# Ensure the timestamp is numeric
df['timestamp'] = pd.to_numeric(df['timestamp'])

# Convert timestamp from milliseconds to seconds and then to datetime
df['date'] = pd.to_datetime(df['timestamp'] // 1000, unit='s')
print(df)

  appVersion platform            userId              eventCode langaugeCode  \
0      8.1.2  Android  92a64c34b7a937df  GOTO_HOME_DIALOG_OPEN        en_US   
1      8.1.2  Android  92a64c34b7a937df  GOTO_HOME_DIALOG_OPEN        en_US   
2      8.1.2  Android  92a64c34b7a937df  GOTO_HOME_DIALOG_OPEN        en_US   
3      8.1.2  Android  92a64c34b7a937df  GOTO_HOME_DIALOG_OPEN        en_US   
4      8.1.2  Android  92a64c34b7a937df  GOTO_HOME_DIALOG_OPEN        en_US   
5      8.1.2  Android  92a64c34b7a937df  GOTO_HOME_DIALOG_OPEN        en_US   
6      8.1.2  Android  92a64c34b7a937df  GOTO_HOME_DIALOG_OPEN        en_US   
7      8.1.3  Android  55484edcf472b02b  GOTO_HOME_DIALOG_OPEN        sk_SK   
8      8.1.3  Android  92a64c34b7a937df  GOTO_HOME_DIALOG_OPEN        en_US   
9      8.1.3  Android  92a64c34b7a937df  GOTO_HOME_DIALOG_OPEN        en_US   

      timestamp                date  
0  1.690082e+12 2023-07-23 03:16:17  
1  1.690082e+12 2023-07-23 03:16:29  
2  1.690082e+12 